In [ ]:
library(biomaRt)
library(dplyr)
library(readr)
library(readxl)
library(venn)
library(ggplot2)
library(sf)
library(ggVennDiagram)

In [ ]:
humanlist <- c("acp2","calsyntenin-1","calsyntenin-2","calsyntenin-3","aplp1","aplp2","app","axl","bcma","betacellulin","betaglycan","car","cd147","cd200","cd43","cd44","cd99","cx3cl1","cxcl16","desmoglein-2","dner","dr6","dystroglycan","e-cadherin","epcam","epha2","epha5","epha7","ephb3","ephb4","ephb6","ephrin-b2","erbb4","f11r","fgfr4","glg1","hla-a2","ifnar2","igf-1r","il11r","il-1r1","il-1r2","il6r","ir","irela","ilrelb","jagged2","kcne2","kcne1","klotho","lar","ldlr","lrp1","lrp1b","lrp6","mer","met","muc1","musk","n-cadherin","neogenin","neurexin-3-b","NLRR3","p75NTR","pianp","plxdc2","pmel17","pdpn","polycystin-1","polyductin","protocadherin-12","ptk7","rage","robo1","sez6","sez6l","sez6l2","sirpa","sorcs1","sorla","sortilin","synedecan-1","synedecan-2","synedecan-3","tie1","tmeff2","tnfr1","trem2","trka","tyro3","vasorin","vegfr3")

In [ ]:
mouselist <- c("apoER2","CACHD1","CADM1","CSF1R","Delta1","dscam","dscaml1","ephb2","ephrin-b2","fgfr3","ghr","l1","nectin1a","nectin3","ng2","notch1","notch2","notch3","notch4","nprc","nradd","pianp","prima","pvrl2","sez6","tkrB","trop2","tyrp1","tyrp2","vgscb1","vgscb2","vgscb3","vgscb4","vldlr")

In [ ]:
mouse_mart <- useEnsembl("ensembl", dataset = "mmusculus_gene_ensembl")
     

In [ ]:
# Get UniProt IDs for these proteins
uniprot_mouse <- getBM(attributes = c("uniprotswissprot", "mgi_symbol"),
                      filters = "external_gene_name",
                      values = mouselist,
                      mart = mouse_mart)

     

mouse_ids <- uniprot_mouse$uniprotswissprot
     

uniprot_mouse <- getBM(attributes = c("uniprotswissprot", "mgi_symbol"),
                      filters = "uniprotswissprot",
                      values = mouse_ids,
                      mart = mouse_mart)


In [ ]:
mouse_genes <- uniprot_mouse$mgi_symbol
     

mapping <- read.delim("HMD_HumanPhenotype.rpt", header = FALSE)
     

colnames(mapping) <- c("Human_Gene","hum_num", "Mouse_Gene", "Mouse_MGI_ID", "HPO_Term","last")
     

result <- mapping %>%
  filter(Mouse_Gene %in% mouse_genes) %>%
  select(Mouse_Gene, Human_Gene)
     

converted_mouse <- result$Human_Gene
     

ensembl <- useEnsembl("ensembl", dataset = "hsapiens_gene_ensembl")

     

# Fetches hgnc_symbol and unirpot ID
results_hum <- getBM(
  attributes = c("hgnc_symbol","uniprotswissprot"), # Attributes: UniProt ID and Gene Symbol
  filters = "hgnc_symbol",                      # Filter by UniProt IDs
  values = humanlist,                                 # Input your list of UniProt IDs
  mart = ensembl                                      # Use the Ensembl mart connection
)
     

human_genes <- results_hum$hgnc_symbol
     

human_genes <-c(human_genes,mouse_genes)
     
     

In [ ]:
ad_genes <- read_excel("AD_genes.xlsx", col_names = FALSE)

In [ ]:
ad_genes <- ad_genes$"...1"

In [ ]:
gene_sets <- list(
  "AD Genes" = ad_genes,
  "Module Genes" = module_genes,
  "PSEN1 substrates" = human_genes
)


In [ ]:

# Create Venn Diagram with custom colors
venn <- ggVennDiagram(
  gene_sets,
  label = "both" # Show both counts and genes
) +
  # Adjust fill colors for better contrast
  scale_fill_gradient(low = "grey", high = "lightblue") +
  # Adjust outline colors for clarity
  scale_color_manual(values = c("black", "black", "black")) +
  # Tweak theme for visibility
  theme(
    legend.position = "none", # Remove legend
    panel.background = element_rect(fill = "white"), # White background
    plot.background = element_rect(fill = "white")  # White plot background
  )
     

In [ ]:
ad_module <- intersect(module_genes,human_genes)
print(ad_module)

In [ ]:
csv_filepaths <- c("oligo_modules/oligoState0.csv", "oligo_modules/oligoState0.csv", "oligo_modules/oligoState0.csv","oligo_modules/oligoState0.csv","oligo_modules/oligoState0.csv","oligo_modules/oligoState0.csv","oligo_modules/oligoState0.csv","oligo_modules/oligoState7 (1).csv","oligo_modules/oligoState8 (1).csv")

In [ ]:
psen1_module <- df$Module[df$Gene == "PSEN1"]
subset_df <- df[df$Module == psen1_module, ]